<a href="https://colab.research.google.com/github/tig1795/word_embeddings_Lexika/blob/main/Bert_bert_base_german_cased_Synonyme.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Imports und Vorbereitungen

In [ ]:
import tensorflow as tf

# Get the GPU device name.
device_name = tf.test.gpu_device_name()

# The device name should look like the following:
if device_name == '/device:GPU:0':
    print('Found GPU at: {}'.format(device_name))
else:
    raise SystemError('GPU device not found')

Found GPU at: /device:GPU:0


In [ ]:
import torch

# If there's a GPU available...
if torch.cuda.is_available():    

    # Tell PyTorch to use the GPU.    
    device = torch.device("cuda")

    print('There are %d GPU(s) available.' % torch.cuda.device_count())

    print('We will use the GPU:', torch.cuda.get_device_name(0))

# If not...
else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")

There are 1 GPU(s) available.
We will use the GPU: Tesla K80


In [ ]:
!pip install transformers

     |████████████████████████████████| 3.5 MB 5.3 MB/s 
     |████████████████████████████████| 596 kB 41.9 MB/s 
     |████████████████████████████████| 67 kB 5.1 MB/s 
     |████████████████████████████████| 895 kB 43.1 MB/s 
     |████████████████████████████████| 6.8 MB 30.8 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


# Laden des Tokenizers und des Modells

In [ ]:
import torch
from transformers import AutoModel, AutoTokenizer

In [ ]:
tokenizer = AutoTokenizer.from_pretrained('bert-base-german-cased')

Downloading:   0%|          | 0.00/29.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/433 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/249k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/474k [00:00<?, ?B/s]

In [ ]:
text = "bank"
marked_text = "[CLS] " + text + " [SEP]"

# Tokenize our sentence with the BERT tokenizer.
tokenized_text = tokenizer.tokenize(marked_text)

# Print out the tokens.
print (tokenized_text)

['[CLS]', 'b', '##ank', '[SEP]']


In [ ]:
indexed_tokens = tokenizer.convert_tokens_to_ids(tokenized_text)

In [ ]:
indexed_tokens

[3, 40, 937, 4]

In [ ]:
segments_ids = [1] * len(tokenized_text)

In [ ]:
# Convert inputs to PyTorch tensors
tokens_tensor = torch.tensor([indexed_tokens])
segments_tensors = torch.tensor([segments_ids])

# Load pre-trained model (weights)
model = AutoModel.from_pretrained('bert-base-german-cased')

# Put the model in "evaluation" mode, meaning feed-forward operation.
model.eval()

Downloading:   0%|          | 0.00/419M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-german-cased were not used when initializing BertModel: ['cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.seq_relationship.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


BertModel(
  (embeddings): BertEmbeddings(
    (word_embeddings): Embedding(30000, 768, padding_idx=0)
    (position_embeddings): Embedding(512, 768)
    (token_type_embeddings): Embedding(2, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): BertEncoder(
    (layer): ModuleList(
      (0): BertLayer(
        (attention): BertAttention(
          (self): BertSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): BertSelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
            (dropout): Dropout(p=0.1, inplace=False)
          

In [ ]:
tokens_tensor

tensor([[  3,  40, 937,   4]])

In [ ]:
# Predict hidden states features for each layer
with torch.no_grad():
    encoded_layers, _ = model(tokens_tensor, segments_tensors)

In [ ]:
encoded_layers

'last_hidden_state'

In [ ]:
from transformers import BertTokenizer, BertModel

import torch

tokenizer = BertTokenizer.from_pretrained('bert-base-german-cased')

model = BertModel.from_pretrained('bert-base-german-cased')

word = "bank"

inputs = tokenizer(word, return_tensors="pt")

outputs = model(**inputs)

word_vect = outputs.pooler_output.detach().numpy()

Some weights of the model checkpoint at bert-base-german-cased were not used when initializing BertModel: ['cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.seq_relationship.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:
w1 = model(**tokenizer("bank", return_tensors="pt")).pooler_output.detach().numpy()
w2 = model(**tokenizer("bank", return_tensors="pt")).pooler_output.detach().numpy()

In [ ]:
w2.shape

(1, 768)

In [ ]:
w1.shape

(1, 768)

# Funktion zur Berechnung der Kosinus-Ähnlichkeit von Wörtern

In [ ]:
from numpy import dot
from numpy.linalg import norm

a = w1
b = w2.T

cos_sim = dot(a, b)/(norm(a)*norm(b))

In [ ]:
cos_sim[0][0]

1.0

In [ ]:
import operator
def foo(w, l):
  w1 = model(**tokenizer(w, return_tensors="pt")).pooler_output.detach().numpy()

  output = {}

  for tw in l:
    w2 = model(**tokenizer(tw, return_tensors="pt")).pooler_output.detach().numpy()
    w2 = w2.T
    cos_sim = dot(w1, w2)/(norm(w1)*norm(w2))
    output[tw] = cos_sim[0][0] # schlauer mit np.squeeze

  return sorted(output.items(), key=lambda kv: kv[1], reverse=True)

# Berechnung der Wort-Ähnlichkeit mit ausgewählten Synoymen mit neuer Bedeutung

## Feminismus

In [ ]:
fem = "Feminismus" 

frau = ["Medizin", "Verweiblichung", "Hormone", "Fettleibigkeit", "Brüste"]

feminismus = foo(fem, frau)

In [ ]:
feminismus

[('Fettleibigkeit', 0.900846),
 ('Hormone', 0.89651287),
 ('Medizin', 0.87313807),
 ('Verweiblichung', 0.86310226),
 ('Brüste', 0.81791043)]

In [ ]:
frau2 = ["Selbstbestimmung", "Bewegung", "Frau", "Gleichberechtigung", "Sexismus"]

feminismus2 = foo(fem, frau2)

In [ ]:
feminismus2

[('Sexismus', 0.92430687),
 ('Gleichberechtigung', 0.9046024),
 ('Selbstbestimmung', 0.8819189),
 ('Bewegung', 0.8682686),
 ('Frau', 0.7646061)]

In [ ]:
frau3 = ["Medizin", "Verweiblichung", "Hormone", "Fettleibigkeit", "Brüste", "Selbstbestimmung", "Bewegung", "Frau", "Gleichberechtigung", "Sexismus"]

feminismus3 = foo(fem, frau3)

In [ ]:
feminismus3

[('Sexismus', 0.92430687),
 ('Gleichberechtigung', 0.9046024),
 ('Fettleibigkeit', 0.900846),
 ('Hormone', 0.89651287),
 ('Selbstbestimmung', 0.8819189),
 ('Medizin', 0.87313807),
 ('Bewegung', 0.8682686),
 ('Verweiblichung', 0.86310226),
 ('Brüste', 0.81791043),
 ('Frau', 0.7646061)]

Ähnliche Ergebnisse wie bei dem Modell mit älteren Texten. Allerdings sind die Werte sowohl für die Wörter zu Feminismus mit alter und neuer Bedeutung wesentlich geringer im Vergleich zum Modell mit älteren Texten.

## Widerstand

In [ ]:
wid = "Widerstand" 

wider = ["Nationalsozialismus", "Nazis", "Hitler", "drittes Reich", "NSDAP"]

widerstand = foo(wid, wider)

In [ ]:
widerstand

[('Nationalsozialismus', 0.92320997),
 ('Hitler', 0.8780154),
 ('Nazis', 0.8689661),
 ('drittes Reich', 0.8657312),
 ('NSDAP', 0.85547894)]

In [ ]:
wider2 = ["Strom", "Elektrizität", "Elektrotechnik", "Ohm", "Elektron"]

widerstand2 = foo(wid, wider2)

In [ ]:
widerstand2

[('Elektron', 0.8149889),
 ('Elektrizität', 0.8107724),
 ('Strom', 0.80551386),
 ('Elektrotechnik', 0.8050134),
 ('Ohm', 0.7847183)]

In [ ]:
wider3 = ["Strom", "Elektrizität", "Elektrotechnik", "Ohm", "Elektron", "Nationalsozialismus", "Nazis", "Hitler", "drittes Reich", "NSDAP"]
widerstand3 = foo(wid, wider3)

In [ ]:
widerstand3

[('Nationalsozialismus', 0.92320997),
 ('Hitler', 0.8780154),
 ('Nazis', 0.8689661),
 ('drittes Reich', 0.8657312),
 ('NSDAP', 0.85547894),
 ('Elektron', 0.8149889),
 ('Elektrizität', 0.8107724),
 ('Strom', 0.80551386),
 ('Elektrotechnik', 0.8050134),
 ('Ohm', 0.7847183)]

Ähnliche Ergebnisse wie bei Feminismus. Allerdings der Wert für Nationalsozialismus ist höher als beim alten und die Werte für die Strombegriffe sind niedriger als bei dem alten Modell. Begriffe des Nationalsozialismus sind wesentlich näher am Wort Widerstand als an Strom. Dies war im alten Modell nicht so --> Indiz für früheres Vorkommen von beiden Bedeutung und heute fast nur noch in Bezug auf Nationalsozialismus? (🥳)

## Batterie

In [ ]:
bat = "Batterie" 
batt = ["Brigade", "Kompanie", "Division", "Infanterie", "Regiment", "Artillerie"]

batterie = foo(bat, batt)

In [ ]:
batterie

[('Brigade', 0.91726667),
 ('Artillerie', 0.9152518),
 ('Infanterie', 0.89203304),
 ('Kompanie', 0.89201987),
 ('Division', 0.863649),
 ('Regiment', 0.853735)]

In [ ]:
batt2 = ["Diesel", "Tank", "Speicher", "Elektronik", "Prototyp", "Laden", "Strom"]

batterie2 = foo(bat, batt2)

In [ ]:
batterie2

[('Tank', 0.94215125),
 ('Speicher', 0.9375412),
 ('Elektronik', 0.92920995),
 ('Prototyp', 0.92912495),
 ('Diesel', 0.8980873),
 ('Strom', 0.8274996),
 ('Laden', 0.8121563)]

In [ ]:
batt3 = ["Diesel", "Tank", "Speicher", "Elektronik", "Brigade", "Kompanie", "Division", "Infanterie", "Regiment", "Artillerie", "Prototyp", "Laden", "Strom"]

batterie3 = foo(bat, batt3)

In [ ]:
batterie3

[('Tank', 0.94215125),
 ('Speicher', 0.9375412),
 ('Elektronik', 0.92920995),
 ('Prototyp', 0.92912495),
 ('Brigade', 0.91726667),
 ('Artillerie', 0.9152518),
 ('Diesel', 0.8980873),
 ('Infanterie', 0.89203304),
 ('Kompanie', 0.89201987),
 ('Division', 0.863649),
 ('Regiment', 0.853735),
 ('Strom', 0.8274996),
 ('Laden', 0.8121563)]

Die ersten vier Wörter sind alle von der Bedeutung Strom. Bei dem alten Modell war es andersrum. Bester Kandidat für Wortbedeutungswandel und Kandidat für Fintuning um zu schauen, ob so die Bedeutung beeinflusst werden kann. 🥳

## Toilette

In [ ]:
toi = "Toilette" 
toil = ["Leinwandtuch", "Ankleidezimmer", "Zimmer", "Körperpflege", "Sichankleiden", "Damenkleidung"]

toilette = foo(toi, toil)

In [ ]:
toilette

[('Damenkleidung', 0.8957899),
 ('Zimmer', 0.88462305),
 ('Leinwandtuch', 0.8794619),
 ('Ankleidezimmer', 0.85786027),
 ('Körperpflege', 0.8506751),
 ('Sichankleiden', 0.8279833)]

In [ ]:
toi = "Toilette" 
toil2 = ["Klo", "WC", "Latrine", "Pissoir", "sanitäre Anlagen"]

toilette2 = foo(toi, toil2)

In [ ]:
toilette2

[('Klo', 0.87802225),
 ('sanitäre Anlagen', 0.8687603),
 ('Latrine', 0.83375996),
 ('Pissoir', 0.82510436),
 ('WC', 0.80534893)]

In [ ]:
toi = "Toilette" 
toil3 = ["Klo", "WC", "Latrine", "Leinwandtuch", "Sichankleiden", "Damenkleidung", "Ankleidezimmer", "Zimmer", "Körperpflege", "Pissoir", "sanitäre Anlagen"]

toilette3 = foo(toi, toil3)

In [ ]:
toilette3

[('Damenkleidung', 0.8957899),
 ('Zimmer', 0.88462305),
 ('Leinwandtuch', 0.8794619),
 ('Klo', 0.87802225),
 ('sanitäre Anlagen', 0.8687603),
 ('Ankleidezimmer', 0.85786027),
 ('Körperpflege', 0.8506751),
 ('Latrine', 0.83375996),
 ('Sichankleiden', 0.8279833),
 ('Pissoir', 0.82510436),
 ('WC', 0.80534893)]

Werte sind generell geringer als beim alten Modell, Werte sind enger zusammen und die Liste spaltet sich nicht so klar wie beim alten Modell. Hier Vermischung der Wortbedeutung, früher ältere Bedeutung klar dominanter.

## Rauchwaren

In [ ]:
rau = "Rauchwaren" 
rauch = ["Pelze", "Pelz", "Pelzwirtschaft", "Fell", "Haar", "Haarkleid"]

rauchwaren = foo(rau, rauch)

In [ ]:
rauchwaren

[('Pelze', 0.9225237),
 ('Haarkleid', 0.84964174),
 ('Haar', 0.8408511),
 ('Fell', 0.83438957),
 ('Pelz', 0.8266047),
 ('Pelzwirtschaft', 0.80043554)]

In [ ]:
rau = "Rauchwaren" 
rauch2 = ["Zigaretten", "Zigarren", "Tabak", "Tabakwaren", "Zigarillo", "Geschäft"]

rauchwaren2 = foo(rau, rauch2)

In [ ]:
rauchwaren2

[('Tabakwaren', 0.97066283),
 ('Zigaretten', 0.9513302),
 ('Tabak', 0.93398064),
 ('Zigarren', 0.88703597),
 ('Zigarillo', 0.88128996),
 ('Geschäft', 0.82955647)]

In [ ]:
rau = "Rauchwaren" 
rauch3 = ["Zigaretten", "Zigarren", "Pelze", "Pelz", "Pelzwirtschaft", "Fell", "Haar", "Haarkleid", "Tabak", "Tabakwaren", "Zigarillo", "Geschäft"]

rauchwaren3 = foo(rau, rauch3)

In [ ]:
rauchwaren3

[('Tabakwaren', 0.97066283),
 ('Zigaretten', 0.9513302),
 ('Tabak', 0.93398064),
 ('Pelze', 0.9225237),
 ('Zigarren', 0.88703597),
 ('Zigarillo', 0.88128996),
 ('Haarkleid', 0.84964174),
 ('Haar', 0.8408511),
 ('Fell', 0.83438957),
 ('Geschäft', 0.82955647),
 ('Pelz', 0.8266047),
 ('Pelzwirtschaft', 0.80043554)]

Eindruck, dass schon früher nur Leute, die in der Pelzwirtschaft tätig waren, das Wort Rauchwaren richtig verwendeten, bleibt bestehen!

## Panzer

In [ ]:
pan = "Panzer" 
panz = ["Schlachtschiff", "Panzerkreuzer", "Kriegsschiff", "Marine", "Fregatte"]

panzer = foo(pan, panz)

In [ ]:
panzer

[('Panzerkreuzer', 0.93746704),
 ('Schlachtschiff', 0.9118255),
 ('Kriegsschiff', 0.9058715),
 ('Marine', 0.88481617),
 ('Fregatte', 0.8257397)]

In [ ]:
pan = "Panzer" 
panz2 = ["gepanzerte Landfahrzeuge", "Kettenfahrzeug", "Kampfwagen", "Kampffahrzeug"]

panzer2 = foo(pan, panz2)

In [ ]:
panzer2

[('Kampfwagen', 0.9190306),
 ('Kampffahrzeug', 0.9096658),
 ('gepanzerte Landfahrzeuge', 0.86047196),
 ('Kettenfahrzeug', 0.82432276)]

In [ ]:
pan = "Panzer" 
panz3 = ["gepanzerte Landfahrzeuge", "Kettenfahrzeug", "Schlachtschiff", "Panzerkreuzer", "Kriegsschiff", "Marine", "Fregatte", "Kampfwagen", "Kampffahrzeug"]

panzer3 = foo(pan, panz3)

In [ ]:
panzer3

[('Panzerkreuzer', 0.93746704),
 ('Kampfwagen', 0.9190306),
 ('Schlachtschiff', 0.9118255),
 ('Kampffahrzeug', 0.9096658),
 ('Kriegsschiff', 0.9058715),
 ('Marine', 0.88481617),
 ('gepanzerte Landfahrzeuge', 0.86047196),
 ('Fregatte', 0.8257397),
 ('Kettenfahrzeug', 0.82432276)]

Die alte Bedeutung von Panzern als Kriegsschiffen scheint auch auf das neue Modell zu zutreffen.

## blöde

In [ ]:
blo = "blöde" 
bloe = ["schüchtern", "zurückhaltend", "zaghaft", "feige", "schwach"]

bloede = foo(blo, bloe)

In [ ]:
bloede


[('feige', 0.94606864),
 ('schüchtern', 0.92043746),
 ('zaghaft', 0.8989982),
 ('zurückhaltend', 0.87314224),
 ('schwach', 0.80933726)]

In [ ]:
blo = "blöde" 
bloe2 = ["dümmlich", "geistlos", "unsinnig", "albern", "blödsinnig"]

bloede2 = foo(blo, bloe2)

In [ ]:
bloede2

[('blödsinnig', 0.9624008),
 ('unsinnig', 0.9510794),
 ('dümmlich', 0.923311),
 ('albern', 0.92264944),
 ('geistlos', 0.86911243)]

In [ ]:
blo = "blöde" 
bloe3 = ["dümmlich", "geistlos", "unsinnig", "schüchtern", "zurückhaltend", "zaghaft", "feige", "schwach", "albern", "blödsinnig"]

bloede3 = foo(blo, bloe3)

In [ ]:
bloede3

[('blödsinnig', 0.9624008),
 ('unsinnig', 0.9510794),
 ('feige', 0.94606864),
 ('dümmlich', 0.923311),
 ('albern', 0.92264944),
 ('schüchtern', 0.92043746),
 ('zaghaft', 0.8989982),
 ('zurückhaltend', 0.87314224),
 ('geistlos', 0.86911243),
 ('schwach', 0.80933726)]

3 der ersten 4 Einträge von der neuen Bedeutung. Leichte Tendenz zur neuen Bedeutung, früher beide eher gleich weit verbreitet?

## wichsen

In [ ]:
wic = "wichsen" 
wich = ["Wichse", "Schuhe", "Schuhcreme", "zum Glänzen bringen", "polieren"]

wichsen = foo(wic, wich)

In [ ]:
wichsen

[('zum Glänzen bringen', 0.83320165),
 ('Wichse', 0.81324416),
 ('Schuhe', 0.7289931),
 ('polieren', 0.7285172),
 ('Schuhcreme', 0.6778083)]

In [ ]:
wic = "wichsen" 
wich2 = ["ornanieren", "masturbieren", "Selbstbefriedigung", "runterholen"]

wichsen2 = foo(wic, wich2)

In [ ]:
wichsen2

[('runterholen', 0.84924155),
 ('ornanieren', 0.83924276),
 ('Selbstbefriedigung', 0.78568345),
 ('masturbieren', 0.730045)]

In [ ]:
wic = "wichsen" 
wich3 = ["ornanieren", "masturbieren", "Wichse", "Schuhe", "Schuhcreme", "zum Glänzen bringen", "polieren", "Selbstbefriedigung", "runterholen"]

wichsen3 = foo(wic, wich3)

In [ ]:
wichsen3

[('runterholen', 0.84924155),
 ('ornanieren', 0.83924276),
 ('zum Glänzen bringen', 0.83320165),
 ('Wichse', 0.81324416),
 ('Selbstbefriedigung', 0.78568345),
 ('masturbieren', 0.730045),
 ('Schuhe', 0.7289931),
 ('polieren', 0.7285172),
 ('Schuhcreme', 0.6778083)]

Heute deutlicher als früher, aber Bedeutung von wichsen eigentlich immer masturbieren

## ausgepowert

In [ ]:
aus = "ausgepowert" 
ausg = ["verarmt", "zahlungsunfähig", "vermögenslos", "bedürftig", "unvermögend"]

ausgepowert = foo(aus, ausg)

In [ ]:
ausgepowert

[('verarmt', 0.90130186),
 ('unvermögend', 0.8465568),
 ('zahlungsunfähig', 0.8436486),
 ('vermögenslos', 0.84248316),
 ('bedürftig', 0.76590776)]

In [ ]:
aus = "ausgepowert" 
ausg2 = ["kraftlos", "ernergielos", "entkräftet", "erschöpft", "ohne Kraft"]

ausgepowert2 = foo(aus, ausg2)

In [ ]:
ausgepowert2

[('ernergielos', 0.9123584),
 ('ohne Kraft', 0.9117893),
 ('entkräftet', 0.9107364),
 ('erschöpft', 0.89362),
 ('kraftlos', 0.8863274)]

In [ ]:
aus = "ausgepowert" 
ausg3 = ["kraftlos", "ernergielos", "entkräftet", "verarmt", "zahlungsunfähig", "vermögenslos", "bedürftig", "unvermögend", "erschöpft", "ohne Kraft"]

ausgepowert3 = foo(aus, ausg3)

In [ ]:
ausgepowert3

[('ernergielos', 0.9123584),
 ('ohne Kraft', 0.9117893),
 ('entkräftet', 0.9107364),
 ('verarmt', 0.90130186),
 ('erschöpft', 0.89362),
 ('kraftlos', 0.8863274),
 ('unvermögend', 0.8465568),
 ('zahlungsunfähig', 0.8436486),
 ('vermögenslos', 0.84248316),
 ('bedürftig', 0.76590776)]

Auf dem älteren Modell nahezu regelmäßiges Abwechseln der Wörter, bei neuem Modell klare Tendenz zu der Bedeutung kraftlos und nicht Armut! Früher beide Bedeutungen heute wieder eher nur noch energielos?

## Eselsbrücke

In [ ]:
ese = "Eselsbrücke" 
esel = ["besondere Schwierigkeit", "Erschwernis", "Hindernis", "Problematik", "Komplikation"]

eselsbrücke = foo(ese, esel)

In [ ]:
eselsbrücke

[('Erschwernis', 0.77906555),
 ('Problematik', 0.77187246),
 ('Hindernis', 0.75601315),
 ('Komplikation', 0.75063276),
 ('besondere Schwierigkeit', 0.7064305)]

In [ ]:
esel2 = ["Hilfsmittel", "Gedächtnisstütze", "Merkhilfe", "Verständnishilfe", "Anhaltspunkt"]

eselsbrücke2 = foo(ese, esel2)

In [ ]:
eselsbrücke2

[('Anhaltspunkt', 0.81152964),
 ('Hilfsmittel', 0.7664819),
 ('Verständnishilfe', 0.7609133),
 ('Merkhilfe', 0.75977206),
 ('Gedächtnisstütze', 0.75782126)]

In [ ]:
esel3 = ["Hilfsmittel", "Gedächtnisstütze", "Merkhilfe", "besondere Schwierigkeit", "Erschwernis", "Hindernis", "Problematik", "Komplikation","Verständnishilfe", "Anhaltspunkt"]

eselsbrücke3 = foo(ese, esel3)

In [ ]:
eselsbrücke3

[('Anhaltspunkt', 0.81152964),
 ('Erschwernis', 0.77906555),
 ('Problematik', 0.77187246),
 ('Hilfsmittel', 0.7664819),
 ('Verständnishilfe', 0.7609133),
 ('Merkhilfe', 0.75977206),
 ('Gedächtnisstütze', 0.75782126),
 ('Hindernis', 0.75601315),
 ('Komplikation', 0.75063276),
 ('besondere Schwierigkeit', 0.7064305)]

Tendenz zur neueren Bedeutung. Wort mit der größten Cosinus Ähnlichkeit gehört zur neuen Bedeutung. Letzten drei Einträge der Liste gehören zur alten Bedeutung. 